In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.utils import shuffle

In [42]:
dataset = pd.read_csv('heart-disease.csv')
dataset = shuffle(dataset)
dataset = np.array(dataset)
X = dataset[:,:-1]
y = dataset[:,-1]

In [43]:
X

array([[41.,  0.,  2., ...,  2.,  0.,  2.],
       [56.,  0.,  0., ...,  1.,  2.,  3.],
       [42.,  1.,  0., ...,  1.,  0.,  1.],
       ...,
       [65.,  1.,  0., ...,  2.,  0.,  3.],
       [57.,  1.,  0., ...,  1.,  1.,  1.],
       [44.,  1.,  2., ...,  2.,  0.,  2.]])

In [44]:
scaler = preprocessing.StandardScaler().fit(X)
scaler

StandardScaler()

In [47]:
X = scaler.transform(X)
X

array([[-6.15858258e+00, -4.62466752e+00,  3.45323773e-02, ...,
        -6.87552016e-01, -1.41421986e+00, -4.62393851e+00],
       [-5.97612820e+00, -4.62466752e+00, -1.84938612e+00, ...,
        -3.32968969e+00,  5.04664034e-01, -1.94760339e+00],
       [-6.14641896e+00, -4.64939962e-03, -1.84938612e+00, ...,
        -3.32968969e+00, -1.41421986e+00, -7.30027363e+00],
       ...,
       [-5.86665558e+00, -4.64939962e-03, -1.84938612e+00, ...,
        -6.87552016e-01, -1.41421986e+00, -1.94760339e+00],
       [-5.96396458e+00, -4.64939962e-03, -1.84938612e+00, ...,
        -3.32968969e+00, -4.54777914e-01, -7.30027363e+00],
       [-6.12209171e+00, -4.64939962e-03,  3.45323773e-02, ...,
        -6.87552016e-01, -1.41421986e+00, -4.62393851e+00]])

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [61]:
print(X_train.shape)
print(y_train.shape)

(227, 13)
(227,)


In [67]:
def init_params():
    W1 = np.random.rand(10, 13) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    W3 = np.random.rand(1, 10) - 0.5
    b3 = np.random.rand(1, 1) - 0.5
    return W1, b1, W2, b2, W3, b3

def ReLU(Z):
    return np.maximum(Z, 0)

def sigmoid(Z):
    return (1/(1 + np.exp(-Z)))

def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = sigmoid(Z3)
    return Z1, A1, Z2, A2, Z3, A3

def ReLU_deriv(Z):
    return 0 > Z

def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y):
    m = Y.shape[0]

    # Backpropagation for the output layer
    dZ3 = A3 - Y
    dW3 = 1/m * dZ3.dot(A2.T)
    db3 = 1/m * np.sum(dZ3, axis=1, keepdims=True)

    # Backpropagation for the second hidden layer
    dA2 = W3.T.dot(dZ3)
    dZ2 = dA2 * ReLU_deriv(Z2)
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)

    # Backpropagation for the first hidden layer
    dA1 = W2.T.dot(dZ2)
    dZ1 = dA1 * ReLU_deriv(Z1)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2, dW3, db3

def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3
    b3 = b3 - alpha * db3

    return W1, b1, W2, b2, W3, b3

In [70]:
def get_predictions(A3):
    return np.where(A3 > 0.5, 1, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    X = X.T
    W1, b1, W2, b2, W3, b3 = init_params()
    
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        
        if i % 100 == 0:
            predictions = get_predictions(A3)
            accuracy = get_accuracy(predictions, Y)
            print(f"Iteration {i}: Accuracy = {accuracy * 100:.2f}%")
    
    return W1, b1, W2, b2, W3, b3

In [80]:
W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, y_train, 0.10, 1000)

Iteration 0: Accuracy = 53.74%
Iteration 100: Accuracy = 64.32%
Iteration 200: Accuracy = 69.16%
Iteration 300: Accuracy = 70.93%
Iteration 400: Accuracy = 72.25%
Iteration 500: Accuracy = 72.69%
Iteration 600: Accuracy = 73.13%
Iteration 700: Accuracy = 74.01%
Iteration 800: Accuracy = 76.21%
Iteration 900: Accuracy = 77.09%
